# Getting sentiment from ChatGPT

In [7]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time
import openai
import redis
import asyncio

In [2]:
# r = redis.Redis("localhost", 5211, decode_responses=True)
db = sqlite3.connect("scraping.db")

In [15]:
cur = db.execute("select * from article where headline != 'None' and content != 'None' and content != '' and feedback is null limit 100")
db.commit()

In [16]:
messages = [{
    "role": "system", 
    "content": "Read the article below, list the named entities that are discussed in it, and choose a sentiment between positive, neutral, and negative for each listed entity separately. Provide your answer in a json format with keys 'name', 'sentiment' and 'explanation'."
}]
replies = {}

with open("../key.txt", "r") as f:
    client = openai.AsyncOpenAI(
        api_key=f.read(),
    )

from tqdm import tqdm

async def get_sentiment(row):
    chat = await client.chat.completions.create( 
        model="gpt-3.5-turbo", 
        messages=messages + [{"role": "user", "content": "Title: " + row[1] + "\nContent: " + row[3]}] 
    )
    reply = chat.choices[0].message.content

    return row[0], reply

async def multiple_sentiments(rows):
    tasks = [get_sentiment(row) for row in rows]
    res = await asyncio.gather(*tasks, return_exceptions=True)
    return res

res = await multiple_sentiments(list(cur))

In [24]:
import json
json.loads(res[1][1])

{'entities': [{'name': 'David Hakhverdyan',
   'sentiment': 'positive',
   'explanation': "He is advocating for the development of a new strategic concept for the agriculture sphere in Armenia and emphasizes the need for the private sector's involvement."},
  {'name': 'Corporative Management High School',
   'sentiment': 'positive',
   'explanation': 'The round table discussion took place at this institution, indicating its commitment to addressing agricultural development issues.'},
  {'name': 'Armenian government',
   'sentiment': 'neutral',
   'explanation': 'Mentioned as a stakeholder that needs to be presented with programs and solutions for agriculture development.'},
  {'name': 'Russian Academy of Sciences',
   'sentiment': 'positive',
   'explanation': 'Academician Abel Aghanbegyan, associated with the Academy, emphasizes the importance of discussions and the creation of development programs for agriculture.'},
  {'name': 'Samvel Avetisyan',
   'sentiment': 'neutral',
   'expla

In [23]:
for r in res:
    db.execute(f'update article set feedback = ? where id = ?', (r[1], r[0]))
db.commit()

In [28]:
cur = db.execute(f'select count(*) from article where content like "%Russia%"')
db.commit()

(l := list(cur))

[(25652,)]

In [30]:
cur = db.execute(f'select count(*) from article where feedback like "%Russia%"')
db.commit()

(l := list(cur))

[(15,)]

In [31]:
cur = db.execute(f'select count(*) from article where headline like "%Russia%"')
db.commit()

(l := list(cur))

[(6659,)]